### Tutorial 6- Pyspark With Python-Introduction To Pyspark Mlib

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read Dataset
training = spark.read.csv('test6.csv', header=True, inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
featureAssembler=VectorAssembler(inputCols=['age','Experience'],outputCol='Independent features')

In [11]:
output = featureAssembler.transform(training)

In [12]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [13]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent features']

In [15]:
finalized_data = output.select("Independent features","Salary")

In [16]:
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [18]:
regressor.coefficients

DenseVector([-90.5483, 1608.7819])

In [19]:
regressor.intercept

16079.136690647425

In [20]:
pred_result=regressor.evaluate(test_data)

In [21]:
pred_result.predictions.show()

+--------------------+------+-----------------+
|Independent features|Salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17214.09079632846|
+--------------------+------+-----------------+



In [22]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(785.909203671541, 617653.2764156357)